In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_csv("C:/Users/agung/Semester 7/Data Sains dan Analisis/UTS/Billionaires Statistics Dataset.csv")

df.head()

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,67059887.0,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640 entries, 0 to 2639
Data columns (total 35 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rank                                        2640 non-null   int64  
 1   finalWorth                                  2640 non-null   int64  
 2   category                                    2640 non-null   object 
 3   personName                                  2640 non-null   object 
 4   age                                         2575 non-null   float64
 5   country                                     2602 non-null   object 
 6   city                                        2568 non-null   object 
 7   source                                      2640 non-null   object 
 8   industries                                  2640 non-null   object 
 9   countryOfCitizenship                        2640 non-null   object 
 10  organization

In [4]:
# Menghitung total nilai NaN dalam setiap kolom
total_nan_per_column = df.isna().sum()

# Menampilkan total nilai NaN per kolom
print(total_nan_per_column)

rank                                             0
finalWorth                                       0
category                                         0
personName                                       0
age                                             65
country                                         38
city                                            72
source                                           0
industries                                       0
countryOfCitizenship                             0
organization                                  2315
selfMade                                         0
status                                           0
gender                                           0
birthDate                                       76
lastName                                         0
firstName                                        3
title                                         2301
date                                             0
state                          

In [5]:
# Karena pada data terdapat kolom yang memiliki kategori boolean maka harus diganti ke numerical. Karena, hal ini berpengaruh terhadap perhitungan nantinya
df['selfMade'] = df['selfMade'].astype(int)

In [7]:
# Memisahkan target (y) dan fitur (X)
y = df['finalWorth']
X = df.drop(['rank', 'finalWorth'], axis=1)

# Menghapus kolom-kolom yang tidak akan digunakan sebagai fitur
cols_to_drop = [
    "source", "industries", "organization", 'title', 'state',
    'residenceStateRegion', "city", "birthDate", 'birthMonth', 'birthDay',
    'lastName', 'firstName', 'birthYear', 'date', 'latitude_country',
    'longitude_country', 'personName', 'gdp_country'
]
X = X.drop(cols_to_drop, axis=1)

X

,category,age,country,countryOfCitizenship,selfMade,status,gender,cpi_country,cpi_change_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country
0,Fashion & Retail,74.0,France,France,0,U,M,110.05,1.1,65.6,102.5,82.5,24.2,60.7,6.705989e+07
1,Automotive,51.0,United States,United States,1,D,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
2,Technology,59.0,United States,United States,1,D,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
3,Technology,78.0,United States,United States,1,U,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
4,Finance & Investments,92.0,United States,United States,1,D,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,Healthcare,51.0,China,China,1,D,M,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09
2636,Food & Beverage,80.0,United States,United States,0,E,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
2637,Manufacturing,60.0,China,China,1,R,M,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09
2638,Real Estate,71.0,China,China,1,D,M,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09


In [8]:
X = X.rename(columns={
    'countryOfCitizenship': 'Citizenship',
    'selfMade': 'Self Made',
    'cpi_change_country': 'CPI Change',
    'cpi_country': 'CPI',
    'gross_tertiary_education_enrollment': 'Gross Tertiary Education',
    'gross_primary_education_enrollment_country': 'Gross Primary Education',
    'life_expectancy_country': 'Life Expectancy',
    'tax_revenue_country_country': 'Tax Revenue',
    'total_tax_rate_country': 'Total Tax',	
    'population_country': 'Population'
})

X

,category,age,country,Citizenship,Self Made,status,gender,CPI,CPI Change,Gross Tertiary Education,Gross Primary Education,Life Expectancy,Tax Revenue,Total Tax,Population
0,Fashion & Retail,74.0,France,France,0,U,M,110.05,1.1,65.6,102.5,82.5,24.2,60.7,6.705989e+07
1,Automotive,51.0,United States,United States,1,D,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
2,Technology,59.0,United States,United States,1,D,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
3,Technology,78.0,United States,United States,1,U,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
4,Finance & Investments,92.0,United States,United States,1,D,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,Healthcare,51.0,China,China,1,D,M,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09
2636,Food & Beverage,80.0,United States,United States,0,E,M,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
2637,Manufacturing,60.0,China,China,1,R,M,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09
2638,Real Estate,71.0,China,China,1,D,M,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09


In [11]:
# Memisahkan nama-nama dari data
names = df['personName']

# Melakukan label encoding untuk kolom 'category' dan 'gender'
label_encoders = {}
categorical_features = ['category', 'gender']

for feature in categorical_features:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature])
    label_encoders[feature] = le

# Melakukan label encoding untuk kolom-kolom lain
features_to_encode = ['country', 'Citizenship', 'Self Made', 'status']

for feature in features_to_encode:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature])
    label_encoders[feature] = le

# Hasil data yang telah di-preprocess
X


,category,age,country,Citizenship,Self Made,status,gender,CPI,CPI Change,Gross Tertiary Education,Gross Primary Education,Life Expectancy,Tax Revenue,Total Tax,Population
0,4,74.0,24,22,0,5,1,110.05,1.1,65.6,102.5,82.5,24.2,60.7,6.705989e+07
1,0,51.0,74,73,1,0,1,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
2,16,59.0,74,73,1,0,1,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
3,16,78.0,74,73,1,5,1,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
4,5,92.0,74,73,1,0,1,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,8,51.0,16,13,1,0,1,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09
2636,6,80.0,74,73,0,1,1,117.24,7.5,88.2,101.8,78.5,9.6,36.6,3.282395e+08
2637,10,60.0,16,13,1,3,1,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09
2638,13,71.0,16,13,1,0,1,125.08,2.9,50.6,100.2,77.0,9.4,59.2,1.397715e+09


In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,  test_size=0.3, random_state=42)

In [13]:
XGBR = XGBRegressor()
XGBR.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
# Prediksi menggunakan data validasi
predictions = XGBR.predict(X_valid)

# Menghitung MAE
mae = mean_absolute_error(y_valid, predictions)

print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 3203.684047506313


In [16]:
XGBR_0 = XGBRegressor(n_estimators=500)
XGBR_0.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [18]:
# Prediksi menggunakan data validasi
predictions = XGBR_0.predict(X_valid)

# Menghitung MAE
mae = mean_absolute_error(y_valid, predictions)

print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 4631.141835289772


In [19]:
XGBR_1 = XGBRegressor(n_estimators=500)
XGBR_1.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [20]:
# Prediksi menggunakan data validasi
predictions = XGBR_1.predict(X_valid)

# Menghitung MAE
mae = mean_absolute_error(y_valid, predictions)

print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 4631.141835289772


In [21]:
# Tentukan range nilai n_estimators yang akan diuji
n_estimators_list = [100, 150, 200, 250, 300]

# Inisialisasi sebuah list untuk menyimpan hasil MAE
mae_results = []

# Loop melalui setiap nilai n_estimators
for n_estimators in n_estimators_list:
    # Inisialisasi model dengan n_estimators tertentu
    model_grid = XGBRegressor(n_estimators=n_estimators)
    
    # Fitting model menggunakan data training
    model_grid.fit(X_train, y_train)
    
    # Lakukan prediksi pada data validasi
    predictions = model_grid.predict(X_valid)
    
    # Hitung Mean Absolute Error
    mae = mean_absolute_error(predictions, y_valid)
    
    # Tampilkan nilai n_estimators dan MAE
    print(f"n_estimators: {n_estimators}, Mean Absolute Error: {mae}")
    
    # Tambahkan nilai MAE ke dalam list
    mae_results.append(mae)

n_estimators: 100, Mean Absolute Error: 4183.687502201759
n_estimators: 150, Mean Absolute Error: 4342.49666607982
n_estimators: 200, Mean Absolute Error: 4438.203753729059
n_estimators: 250, Mean Absolute Error: 4499.314507961273
n_estimators: 300, Mean Absolute Error: 4535.3653367119605


In [22]:
XGBR_2 = XGBRegressor(n_estimators=100)
XGBR_2.fit(X_train, y_train,
               early_stopping_rounds=5,
               eval_set=[(X_valid, y_valid)],
               verbose=False)

predictions_2 = XGBR_2.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions_2, y_valid)))

Mean Absolute Error: 3175.277682834201


c:\Users\agung\anaconda3\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
